#### Test on convert a netCDF file to a GeoTIFF

#### To test then in a Leaflet map

 * https://github.com/stuartmatthews/leaflet-geotiff
 * https://mygeodata.cloud/converter/

In [1]:
import numpy as np
import netCDF4
from osgeo import osr
from osgeo import gdal

In [2]:
# 
import os
'GDAL_DATA' in os.environ 

True

In [3]:
# temp.nc has been produced from levitus_climatology with ferret
# yes? use levitus_climatology
# yes? save/clobber/file=temp.nc temp[k=1]

infile = 'uwnd.nc'

ncfile = netCDF4.Dataset(infile, 'r')
ncfile.variables.keys()

[u'FNOCX', u'FNOCY', u'TIME', u'UWND']

In [4]:
dims = ncfile.dimensions.keys()
dims

[u'FNOCX', u'FNOCY', u'TIME']

In [5]:
lon = ncfile.variables[dims[0]][:]
lat = ncfile.variables[dims[1]][:]
var = ncfile.variables['UWND'][0][:]

print var.shape

(73, 144)


In [6]:
nrows,ncols = np.shape(var)
xres = 360./ncols
yres = 180./nrows

# Retrieve edges considering regular axis
xmin,xmax,ymin,ymax = [lon[0]-xres/2,lon[-1]+xres/2,lat[0]-yres/2,lat[-1]+yres/2]

print nrows,ncols
print xmin,xmax,ymin,ymax
print xres, yres

73 144
18.75 378.75 -91.2328767123 91.2328767123
2.5 2.46575342466


In [24]:
# https://stackoverflow.com/questions/43254024/extracting-specific-netcdf-info-and-converting-to-geotiff-in-python

In [7]:
outfile = 'uwnd.tiff'

driver = gdal.GetDriverByName('GTiff')

dst_ds = driver.Create(outfile, ncols, nrows, 
                       bands=1, eType=gdal.GDT_Float32, options=['COMPRESS=LZW'])

geotransform=(xmin,xres,0,ymax,0,-yres)   
# That's (top left x, w-e pixel resolution, rotation (0 if North is up), 
#         top left y, rotation (0 if North is up), n-s pixel resolution)

dst_ds.SetGeoTransform(geotransform)     # specify coords
srs = osr.SpatialReference()             # establish encoding
srs.ImportFromEPSG(4326)                 # WGS84 lat/long
dst_ds.SetProjection(srs.ExportToWkt())   # export coords to file

dst_ds.SetMetadata({'key1': '1', 'key2': 'yada'})

band = dst_ds.GetRasterBand(1)
band.WriteArray(var[::-1,:])            # reverse the array

band.SetDescription('blabla')
band.SetMetadata({'name': 'AAAAAA', 'key2': 'BBBBB'})
band.SetNoDataValue(-1E10)
                
# Close properly the dataset
dst_ds = None

In [8]:
test1 = osr.SpatialReference()
test1.ImportFromEPSG(4326)

print test1.ExportToPrettyWkt()

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]


In [9]:
# Check with
! gdalinfo -mm uwnd.tiff

Driver: GTiff/GeoTIFF
Files: uwnd.tiff
Size is 144, 73
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (18.750000000000000,91.232876712328761)
Pixel Size = (2.500000000000000,-2.465753424657534)
Metadata:
  AREA_OR_POINT=Area
  key1=1
  key2=yada
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (      18.750,      91.233) ( 18d45' 0.00"E, 91d13'58.36"N)
Lower Left  (  18.7500000, -88.7671233) ( 18d45' 0.00"E, 88d46' 1.64"S)
Upper Right (     378.750,      91.233) (Invalid angle, 91d13'58.36"N)
Lower Right (     378.750,     -88.767) (Invalid angle, 88d46' 1.64"S)
Center      (     198.750,       1.233) (198d45' 0.00"E,  1d13'58.36"N)
Band 1 Block=144x14 Type=Float32, ColorInterp=Gray
  Description = blabl